## 가져올 정보
1. 코디셋 이미지
2. 태그
2. 코디 아이템별 내용
    - 제품명
    - 브랜드
    - 좋아요
    - 제품 이미지

In [14]:
import pandas as pd
import numpy as np
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import re
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome()

capabilities = driver.capabilities

data_list = []
items_list = []

def musinsa_collector(url):
    driver.get(url)
    time.sleep(2)
    
    # 여자
    # driver.find_element(By.CSS_SELECTOR, '#footerCommonPc > div.sc-pvystx-0.iRAZYo > button:nth-child(3)').click()
    # 남자
    driver.find_element(By.CSS_SELECTOR, '#footerCommonPc > div.sc-pvystx-0.iRAZYo > button:nth-child(2)').click()
    time.sleep(2) 
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 코디 리스트 가져오기
    codi_list = []
    codi_list = [tag['onclick'].split("'")[1] for tag in soup.find_all('a', {'class': 'style-list-item__link'})]
    time.sleep(5)
    print(codi_list)

    items_list = []

    # 코디 개별 url 들어가기
    for i in codi_list:
        codi_url = f'https://www.musinsa.com/app/codimap/views/{i}?style_type=americancasual&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1'
        driver.get(codi_url)
        time.sleep(3)
        codi = {}
        # 코디 카테고리
        codi['codi_name'] = style_type

        # 코디셋 이미지
        codi['codi_img'] = driver.find_element(By.XPATH, '//img[@class="photo"]').get_attribute('src')
        
        # 태그
        tag = driver.find_elements(By.XPATH, '//*[@id="style_info"]/div[1]/div/a')
        # 태그 텍스트 추출하여 리스트에 저장
        codi['tags'] = [tag.text for tag in tag]

        # 코디 상품 리스트 (여기부터 수정!!!!!!!!!!!!!!!)
        codi_item = driver.find_elements(By.XPATH, '//a[@class="styling_img"]')

        # 각 a 태그에서 href 속성 값 추출하여 data_no 리스트에 추가
        data_no_list = []
        for a_tag in codi_item:
            href_value = a_tag.get_attribute("href")
            data_no_list.append(int(href_value.split("/")[-1]))
            
        codi['data_no_list'] = data_no_list

        items_list_per_codi = []

        for d in data_no_list:
            product_url = f"https://www.musinsa.com/app/goods/{d}"
            driver.get(product_url)

            # Create a new soup object with the updated HTML content
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            time.sleep(3)
            
            item = {}
            item['data_no'] = d
            
            item['big_category'] = soup.find('a', class_='product-detail__sc-up77yl-1 hYzMBL').text if soup.find('a', class_='product-detail__sc-up77yl-1 hYzMBL') else None
            print(item['big_category'])

            # 중분류 수정해야함!
            item['mid_category'] = driver.find_element(By.CSS_SELECTOR, '#product-top > div > div.product-detail__sc-up77yl-0.htaeEr > a:nth-child(3)').get_attribute('title')
            print(item['mid_category'])

            item['product_name'] = soup.find('h3', class_='product-detail__sc-1klhlce-3 fjguJZ').text if soup.find('h3', class_='product-detail__sc-1klhlce-3 fjguJZ') else None
            print(item['product_name'])

            item['brand'] = soup.find('a', class_='product-detail__sc-achptn-9 dWHdxq').text if soup.find('a', class_='product-detail__sc-achptn-9 dWHdxq') else None
            print(item['brand'])

            item['like'] = soup.find('span', class_='product-detail__sc-achptn-4 flUHrZ').text if soup.find('span', class_='product-detail__sc-achptn-4 flUHrZ') else None
            print(item['like'])

            try:
                item['price'] = soup.find('span', class_='product-detail__sc-1p1ulhg-7 gQybFn').text if soup.find('span', class_='product-detail__sc-1p1ulhg-7 gQybFn') else None
            except NoSuchElementException:
                item['price'] = soup.find('span', class_ = 'product-detail__sc-1p1ulhg-7 kpTJXD').text if soup.find('span', class_='product-detail__sc-1p1ulhg-7 kpTJXD') else None
            print(item['price'])

            # item['product_img'] = driver.find_element(By.XPATH, '//*[@id="bigimg"]').get_attribute('src')
            try:
                item['product_img'] = driver.find_element(By.XPATH, '//*[@id="bigimg"]').get_attribute('src')
            except StaleElementReferenceException:
                # Handle stale element reference exception (e.g., re-locate the element)
                pass
            print(item['product_img'])
        
            items_list_per_codi.append(item)
            
        codi['items_list'] = items_list_per_codi
        data_list.append(codi)
        items_list.append(items_list_per_codi)        
    print(codi)


# 여자/ 조회순
# style_types = ['sports', 'street', 'gorpcore']
# ['americancasual','casual', 'chic', 'formal', 'girlish', 'golf', 'homewear', 'retro', 'romantic', 'sports', 'street', 'gorpcore']
# 남자/ 조회순
style_types = ['sports', 'street', 'gorpcore']
# ['americancasual','casual', 'chic', 'dandy', 'formal', 'golf', 'homewear', 'sports', 'street', 'gorpcore']
url = 'https://www.musinsa.com/app/codimap/lists?style_type={}&tag_no=&brand=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'

for style_type in style_types:
    url = url.format(style_type)
    musinsa_collector(url)
    print(f'Style Type: {style_type}')

codi_df = pd.DataFrame(data_list)

csv_file_path = 'codimap_men02.csv'
codi_df.to_csv(csv_file_path, index = False, encoding = 'utf-8')

driver.quit()



['5886', '8008', '287', '7020', '646', '4693', '14233', '17750', '7082', '6678', '2836', '9351', '6206', '10874', '19183', '7991', '16579', '19060', '15125', '8412', '8036', '6050', '3745', '8272', '14758', '8822', '5803', '12868', '19486', '13121', '10998', '15073', '1542', '14696', '19537', '5302', '15881', '8037', '10679', '3124', '6741', '13953', '7563', '5260', '5598', '19745', '14349', '16853', '15309', '4491', '2756', '13708', '13201', '14560', '19441', '1243', '4637', '15931', '5308', '20227']
바지
트레이닝/조거 팬츠
락업 트랙팬츠 - 올리브그린 / H41386
ADIDAS
1,917
None
https://image.msscdn.net/images/goods_img/20210709/2025409/2025409_1_500.jpg
스포츠/용품
스포츠가방
더플백-라이트니스 에디션
AMBITIONEM
211
None
https://image.msscdn.net/images/goods_img/20200722/1523632/1523632_1_500.png
스니커즈
패션스니커즈화
바트나 러너 화이트
EARTHUS
514
159,000원
https://image.msscdn.net/images/goods_img/20200102/1261512/1261512_1_500.jpg
스니커즈
패션스니커즈화
BW 아미 - 화이트 / BZ0579
ADIDAS
70,766
139,000원
https://image.msscdn.net/images/goods_img/20170615/57763

In [15]:
codi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   codi_name     180 non-null    object
 1   codi_img      180 non-null    object
 2   tags          180 non-null    object
 3   data_no_list  180 non-null    object
 4   items_list    180 non-null    object
dtypes: object(5)
memory usage: 7.2+ KB


In [16]:
codi_df.head(10)

,codi_name,codi_img,tags,data_no_list,items_list
0,sports,https://image.msscdn.net/images/codimap/detail...,"[#편안한, #애슬레저룩, #스포티, #로고플레이, #컬러블록, #여름, #스포츠,...",[],[]
1,sports,https://image.msscdn.net/images/codimap/detail...,"[#스포티, #로고플레이, #편안한, #애슬레저룩, #플리스, #가을, #스포츠, ...","[2025409, 1523632, 1261512]","[{'data_no': 2025409, 'big_category': '바지', 'm..."
2,sports,https://image.msscdn.net/images/codimap/detail...,"[#가을, #스포츠, #아디다스, #리, #래쉬, #패션스니커즈화, #트레이닝 재킷...","[577636, 1539028]","[{'data_no': 577636, 'big_category': '스니커즈', '..."
3,sports,https://image.msscdn.net/images/codimap/detail...,"[#스포티, #편안한, #로고플레이, #가을, #스포츠, #브룩스, #나이키, #론...",[],[]
4,sports,https://image.msscdn.net/images/codimap/detail...,"[#가을, #스포츠, #페플, #노스페이스, #나이키, #반소매 티셔츠, #숏패딩/...","[1250651, 1524283]","[{'data_no': 1250651, 'big_category': '상의', 'm..."
5,sports,https://image.msscdn.net/images/codimap/detail...,"[#스포티, #편안한, #로고플레이, #모노톤, #여름, #스포츠, #네이키드니스,...","[581419, 1744960]","[{'data_no': 581419, 'big_category': '가방', 'mi..."
6,sports,https://image.msscdn.net/images/codimap/detail...,"[#스포티, #셋업코디, #애슬레저룩, #모노톤, #콜라보, #레터링, #봄, #스...","[1571879, 1750044, 825978]","[{'data_no': 1571879, 'big_category': '아우터', '..."
7,sports,https://image.msscdn.net/images/codimap/detail...,"[#스포티, #레저스포츠, #수영, #모노톤, #그래픽, #로고플레이, #컬러블록,...","[2635785, 2579369, 995895, 1523150, 1514307]","[{'data_no': 2635785, 'big_category': '스포츠/용품'..."
8,sports,https://image.msscdn.net/images/codimap/detail...,"[#애슬레저룩, #컬러블록, #로고플레이, #편안한, #스포티, #가을, #스포츠,...","[2100351, 1306364]","[{'data_no': 2100351, 'big_category': '스포츠/용품'..."
9,sports,https://image.msscdn.net/images/codimap/detail...,"[#모노톤, #스포티, #편안한, #애슬레저룩, #로고플레이, #카고, #가을, #...","[1404195, 1690349, 1825859, 1171117]","[{'data_no': 1404195, 'big_category': '아우터', '..."
